In [ ]:
#import base pacakges
import numpy as np
# import matplotlib.pyplot as plt
# bokeh imports
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import ColorBar, LinearColorMapper,BasicTicker, ColumnDataSource, Slider, CustomJS
from bokeh.palettes import Viridis256, Inferno256, Cividis256
from bokeh.layouts import row, gridplot
# progress bar and delay
from tqdm.notebook import tqdm
import time

In [ ]:
# Define Physical constants
c=3E11 # in mm/s

# Define Input Paramters
ω_0_in=7.2 # in mm
d_in=450 # in mm
freq=95E9
# elliptical mirror
R1=1
R2=1
# extracted params==>
foc=339.5
λ=c/freq


In [ ]:
# function for calc. transversile profile
def cal_trans_prof(z,ω_0,λ): ### λ aprrox by λ=c/f
    z_r=np.pi*ω_0**2/λ
    return    ω_0*np.sqrt(1+(z/z_r)**2)

In [ ]:
# function for calc. d_out and w_0_out
def cal_ω_0_out_and_d_out(d_in,ω_0_in,foc,λ):
    C=-1/foc
    z_c=np.pi*ω_0_in**2/λ
    d_out=-(d_in*(C*d_in+1)+C*z_c**2)/((C*d_in+1)**2+C**2*z_c**2)
    ω_0_out=ω_0_in/np.sqrt((C*d_in+1)**2+C**2*z_c**2)
    return d_out,ω_0_out

In [ ]:
# results d_out and w_0_out:
d_out,ω_0_out = cal_ω_0_out_and_d_out(d_in,ω_0_in,foc,λ)
print("The resulting parameters are: \nd_out= {}, ω_0_out= {}".format(d_out,ω_0_out))

In [ ]:
# function that creates beam waist plot
def create_plot(X, d_in=d_in,ω_0_in=ω_0_in,foc=foc,λ=λ,d_out=None,ω_0_out=None,):
    if (("d_out" and "ω_0_out" in locals()) and (d_out==None or ω_0_out==None))==True: #measure to save compute
        d_out,ω_0_out = cal_ω_0_out_and_d_out(d_in,ω_0_in,foc,λ)
    
    if (isinstance(X, int)==True or isinstance(X, float)==True)==False:
        res=np.copy(X)
        for i,x in enumerate(res):
            if x<=d_in:
                res[i]=cal_trans_prof(x,ω_0_in,λ)
            else:
                res[i]=cal_trans_prof((d_out+d_in)-x,ω_0_out,λ)
    else:
        if X<=d_in:
            res=cal_trans_prof(X,ω_0_in,λ)
        else:
            res=cal_trans_prof((d_out+d_in)-X,ω_0_out,λ)
    
    
    return res

In [ ]:
# plot waist
z_vals=np.linspace(0,d_in+2*d_out,1000)
r_vals=create_plot(X=z_vals, d_in=d_in,ω_0_in=ω_0_in,foc=foc,λ=λ)

output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
fig = figure(title="beam width", x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
             plot_width=500, plot_height=300, tools=tools,active_drag=None,tooltips=[("(z , r)", "($x , $y)")])
# plot beam
fig.line(z_vals,r_vals, line_width=2, color="blue")
fig.line(z_vals,-r_vals, line_width=2, color="blue")
# show
show(fig)


In [ ]:
import numpy as np

output_notebook()

from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, output_file, show

x=np.linspace(0,d_in+2*d_out,1000)
y=create_plot(X=x, d_in=450,ω_0_in=ω_0_in,foc=foc,λ=c/(95E9))
z=-create_plot(X=x, d_in=450,ω_0_in=ω_0_in,foc=foc,λ=c/95E9) #for negative values

source = ColumnDataSource(data=dict(x=x, y=y, z=z))

plot = figure(y_range=(-100,100),plot_height=300,plot_width=600,title="beam width", 
              x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
              tools=tools,active_drag=None,tooltips=[("(z , r)", "($x , $y)")])

plot.line('x', 'y', source=source, line_width=3, color="blue")
plot.line('x', 'z', source=source, line_width=3, color="blue")

z_pos_slider = Slider(start=330, end=700, value=450, step=10, title="z-pos of mirror in mm")
freq_slider = Slider(start=70, end=120, value=95, step=1, title="frequency in GHz")

callback = CustomJS(args=dict(source=source,c=c,ω_0_in=ω_0_in,foc=foc, z_pos=z_pos_slider, freq=freq_slider),
                    code="""
    const data = source.data;
    const d_in = z_pos.value;
    const frequency = freq.value;
    const x = data['x']
    const y = data['y']
    const z = data['z']
    
    C=-1/foc
    λ=c/(frequency*1E9)
    z_c=Math.PI*ω_0_in**2/λ;
    d_out=-(d_in*(C*d_in+1)+C*z_c**2)/((C*d_in+1)**2+C**2*z_c**2);
    ω_0_out=ω_0_in/((C*d_in+1)**2+C**2*z_c**2)**(1/2);
    
    for (var i = 0; i < x.length; i++) {
        if (x[i]<=d_in) {
        z_r=Math.PI*ω_0_in**2/λ;
            y[i]=ω_0_in*(1+(x[i]/z_r)**2)**(1/2);
            z[i]=-y[i];} 
        else {
        z_r=Math.PI*ω_0_out**2/λ;
            y[i]=ω_0_out*(1+((d_in+d_out-x[i])/z_r)**2)**(1/2);
            z[i]=-y[i];} 
    }
    source.change.emit();
""")

z_pos_slider.js_on_change('value', callback)
freq_slider.js_on_change('value', callback)

layout = row(plot,column(z_pos_slider, freq_slider))


show(layout)

In [ ]:
# creating canvas
max_r=100 #in mm
max_z=np.int((np.round(d_in+1.5*d_out))) # in mm
canvas_size=(max_r*2+1,max_z) #uneven for symmetry
canvas=np.zeros(canvas_size)

In [ ]:
# calculation of beam intensity
def calculate_beam(r,ω_0,ω_z):
    return ((ω_0/ω_z)**2*np.exp(-2*r**2/ω_z**2))

# calculation plot
def calculate(λ=λ,d_out=d_out,ω_0_out=ω_0_out,d_in=d_in,ω_0_in=ω_0_in,canvas=canvas):
    # calculations
    just=(canvas.shape[0]-1)/2 #justify
    # very naive intensity for post reflection beam
    I_cell=(cal_trans_prof(d_in,ω_0_in,λ)/cal_trans_prof(-d_out,ω_0_out,λ)*ω_0_in/ω_0_out)**2
    image=np.copy(canvas)
    for z in tqdm(range(0,canvas.shape[1])):
        for r in range(0,canvas.shape[0]):
            if z<=d_in:
                ω_z=cal_trans_prof(z,ω_0_in,λ)
                image[r,z]=calculate_beam((r-just),ω_0_in,ω_z)
            else:
                ω_z=cal_trans_prof(z-round(d_in+d_out),ω_0_out,λ)
                image[r,z]=calculate_beam((r-just),ω_0_out,ω_z)*I_cell
    return image

In [ ]:
# calculate image
image=calculate()

In [ ]:
# plot
z_vals=np.linspace(0,d_in+2*d_out,1000)
r_vals=create_plot(X=z_vals, d_in=d_in,ω_0_in=ω_0_in,foc=foc,λ=λ)

# plot image
output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
# colorbar
color_mapper = LinearColorMapper( palette=Cividis256, low=0, high=1)
plot = figure(x_range=(-0.5,image.shape[1]-0.5), y_range=(-image.shape[0]/2,image.shape[0]/2),plot_width=600, plot_height=300,
             title="beam intensity at given params", x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
             tools=tools, active_drag=None, tooltips=[("(z , r)", "($x , $y)"), ("Intensity", "@image")])
plot.image(image=[image], color_mapper=color_mapper,dh=[image.shape[0]], dw=[image.shape[1]], x=[-0.5], y=[-image.shape[0]/2])

# show beam width
plot.line(z_vals,r_vals, line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
plot.line(z_vals,-r_vals, line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
# color bar
color_bar = ColorBar(color_mapper=color_mapper,label_standoff=12, location=(0,0))
plot.add_layout(color_bar, 'right')
# legend
plot.legend.label_text_font_size = '10pt'
plot.legend.background_fill_alpha = 0.5
# show
show(plot)

# Calc range of frequencies 70 to 110 GHz

In [ ]:
# creating canvas
max_r=100 #in mm
max_z=np.int((np.round(d_in+1.5*d_out))) # in mm
canvas_size=(max_r*2+1,max_z) #uneven for symmetry
canvas=np.zeros(canvas_size)

In [ ]:
# Calculate

#define variables
frequencies=np.arange(70,115,5)*1E9
c=3E11
wavelengths=c/frequencies
# include altered ω_cell
ω_0_in_vals=[7.85,7.67,7.47,7.26,7.03,7.16,7.23,7.26,7.24]
d_in_val=d_in
# save values for later use:
saved = []
beam_saved=[]
beam_vals=np.zeros((len(frequencies),len(z_vals),len(z_vals)))
print("total progress:")
for i,lam in tqdm(enumerate(wavelengths),total=len(wavelengths)):
    print("generating image {} of {}:".format(i+1,len(ω_0_in_vals)))
    d_out,ω_0_out = cal_ω_0_out_and_d_out(d_in_val,ω_0_in_vals[i],foc,lam)
    saved.append(calculate(λ=lam,d_out=d_out,ω_0_out=ω_0_out,d_in=d_in_val,ω_0_in=ω_0_in_vals[i],canvas=canvas))
    z_vals=np.linspace(0,d_in+2*d_out,1000)
    r_vals=create_plot(X=x, d_in=450,ω_0_in=ω_0_in,foc=foc,λ=lam,d_out=d_out,ω_0_out=ω_0_out) 
    beam_saved.append([z_vals,r_vals])

In [ ]:
from bokeh.layouts import gridplot

color_mapper = LinearColorMapper( palette=Cividis256, low=0, high=1)
tools = "crosshair,pan,wheel_zoom,reset,save"
plots = [figure(x_range=(-0.5,canvas.shape[1]-0.5), y_range=(-canvas.shape[0]/2,canvas.shape[0]/2),plot_width=330, plot_height=200,
             title="Frequency= {}GHz".format(np.int(frequencies[i]/1E9)), x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
             tools=tools, tooltips=[("(z , r)", "($x , $y)"), ("Intensity", "@image")]) for i in range(len(saved))]
for i,plot in enumerate(plots):
    plot.add_layout(color_bar, 'right')
    plot.image(image=[saved[i]], color_mapper=color_mapper,dh=[image.shape[0]], dw=[image.shape[1]], x=[-0.5], y=[-image.shape[0]/2])
    # beam profile
    plot.line(beam_saved[i][0],beam_saved[i][1], line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
    plot.line(beam_saved[i][0],-beam_saved[i][1], line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
    # legend
    plot.legend.label_text_font_size = '7pt'
    plot.legend.background_fill_alpha = 0.5
    plot.legend.padding=2
    glyphs = [plot]
color_bar = ColorBar(color_mapper=color_mapper,label_standoff=12, location=(0,0))
show(gridplot(children = plots, ncols = 3, merge_tools = True))

# Project: Gaussian Beam Optimizer

In [ ]:
# calculates plot from d_in to end of canvas, implemented to save compute
def int_opt_chamber_mapper(r_chamber,d_in=d_in,canvas=canvas,ω_0_in=ω_0_in,foc=foc,λ=λ):
    radius=np.int(np.round(r_chamber))
    canvas_shape=canvas.shape
  # params
    image=np.copy(canvas)
    just=(canvas_shape[0]-1)/2 
    d_out,ω_0_out = cal_ω_0_out_and_d_out(d_in,ω_0_in,foc,λ)
    I_cell=(cal_trans_prof(d_in,ω_0_in,λ)/cal_trans_prof(-d_out,ω_0_out,λ)*ω_0_in/ω_0_out)**2
    mid=np.int(np.round(just))
    # plot only rectangle
    for z in range(np.int(np.round(d_in)),canvas_shape[1]):
        for r in range(mid-radius,mid+radius+1):
            ω_z=cal_trans_prof(z-round(d_in+d_out),ω_0_out,λ)
            image[r,z]=calculate_beam((r-just),ω_0_out,ω_z)*I_cell
    return image

In [ ]:
def calculate_int_chamber(pos_chamber,r_chamber,image=image):
    # check size chamber
    image_shape=image.shape
    pos=(np.int(np.round((image_shape[0]-1)/2)),np.int(np.round(pos_chamber)))
    radius=np.int(np.round(r_chamber))
    if (pos[0]+radius)>=image_shape[0] or (pos[0]-radius)<0 or (pos[1]+radius)>=image_shape[1] or (pos[1]-radius)<0:
        raise Exception("Sorry, chamber too large. Use larger canvas or decrease r_chamber.") 
    # params
    summ=0
    # sum circle
    for z in range(pos[1]-radius,pos[1]+radius+1):
        for r in range(pos[0]-radius,pos[0]+radius+1):
            if np.sqrt((pos[0]-r)**2+(pos[1]-z)**2)<=radius:
                summ=summ+image[r,z]
    return summ

In [ ]:
# demonstrate int chamber for fixed d_in
X=range(550,2000,50)
Y=np.copy(X)
image=int_opt_chamber_mapper(100)
for i,x in tqdm(enumerate(X),total=len(X)):
    Y[i]=calculate_int_chamber(x,100,image)
output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
fig = figure(title="Intensity at chamber position", x_axis_label='chamber position in [mm]',y_axis_label='Intensity', 
             plot_width=500, plot_height=300, tools=tools,active_drag=None,tooltips=[("(z , r)", "($x , $y)")])
# plot beam
fig.line(X,Y, line_width=2, color="blue")
# show
show(fig)

In [ ]:
def opt_image_generator(Z_vals,canvas,r_chamber=100,ω_0_in=ω_0_in,foc=foc,λ=λ):
    # check if Z_vals fits
    if Z_vals[0]<0 or Z_vals[-1]>=canvas.shape[1]:
        raise Exception("Z_vals exceed canvas. Adjust it to canvas size") 
    radius=np.int(np.round(r_chamber))
    params=np.full((len(Z_vals)-1, len(Z_vals[Z_vals>(np.round(Z_vals[0])+np.round(radius))])),-1)
    first_d_in=Z_vals[0]
    first_pos_chamber=Z_vals[Z_vals>(np.round(first_d_in)+np.round(radius))][0]
    #calculate in grid
    for i,d_in in tqdm(enumerate(Z_vals),total=len(Z_vals)):
        image=int_opt_chamber_mapper(r_chamber,d_in=d_in,canvas=canvas,ω_0_in=ω_0_in,foc=foc,λ=λ)
        pos_vals=Z_vals[Z_vals>(np.round(d_in)+np.round(radius))]
        for j,pos_chamber in enumerate(pos_vals):
            params[i,j+i]=calculate_int_chamber(pos_chamber,r_chamber,image=image)
    return params, (first_d_in,first_pos_chamber)

In [ ]:
# calculate optimization plot
Z_vals=np.arange(300,2000,100) 
print("approximate total ca0lc. time= {:.1f} min".format(len(Z_vals)*(7/60)))
image, first =opt_image_generator(Z_vals,canvas,r_chamber=50)
pos_chamber_vals=Z_vals[Z_vals>=first[1]]

In [ ]:
# plot
# plot image
output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
# colorbar
color_mapper = LinearColorMapper( palette=Cividis256, low=-1, high=np.amax(image))
plot = figure(x_range=(pos_chamber_vals[0]-(pos_chamber_vals[1]-pos_chamber_vals[0])/2,pos_chamber_vals[-1]-(pos_chamber_vals[1]-pos_chamber_vals[0])/2), 
              y_range=(Z_vals[0]-(Z_vals[1]-Z_vals[0])/2,Z_vals[-1]-(Z_vals[1]-Z_vals[0])/2),plot_width=600, plot_height=300,
              title="Intensity in chamber by chamber_pos and mirror pos", x_axis_label='chamber pos in [mm]',y_axis_label='mirror pos [mm]', 
              tools=tools, active_drag=None, tooltips=[("(z , r)", "($x , $y)"), ("Intensity", "@image")])
plot.image(image=[image], color_mapper=color_mapper, dh=[Z_vals[-1]-Z_vals[0]], dw=[pos_chamber_vals[-1]-pos_chamber_vals[0]],
           x=[pos_chamber_vals[0]-(pos_chamber_vals[1]-pos_chamber_vals[0])/2], y=[Z_vals[0]-(Z_vals[1]-Z_vals[0])/2])

# color bar
color_bar = ColorBar(color_mapper=color_mapper,label_standoff=12, location=(0,0))
plot.add_layout(color_bar, 'right')
# show
print("     In the following plot values mit intensity= -1 are empty")
plot.yaxis.ticker = Z_vals[::10]
plot.xaxis.ticker = pos_chamber_vals[::10]
show(plot)

In [ ]:
# show max results
print("The maximum intensity of {} in arbitraty units is given for:".format(np.amax(image)))
opt_pos=np.unravel_index(image.argmax(), image.shape)
print("mirror position= {} mm, chamber position= {} mm".format(Z_vals[opt_pos[0]],pos_chamber_vals[opt_pos[1]]))
opt_coords=(Z_vals[opt_pos[0]],pos_chamber_vals[opt_pos[1]])

In [ ]:
# creating canvas
max_r=150 #in mm
max_z=np.int((np.round(d_in+1.5*d_out))) # in mm
canvas_size=(max_r*2+1,max_z) #uneven for symmetry
canvas=np.zeros(canvas_size)

In [ ]:
# calculate image
r_chamber=50
illust=np.copy(canvas)
# plot chamber into image
radius=np.int(np.round(r_chamber))
pos=(np.int(np.round(illust.shape[0]-1)/2),np.int(np.round(opt_coords[1])))
for i in range(pos[0]-radius,pos[0]+radius+1):
    for j in range(pos[1]-radius,pos[1]+radius+1):
        if np.sqrt((pos[0]-i)**2+(pos[1]-j)**2)<=radius:
            illust[i,j]=1

In [ ]:
# plot optimized posiions
z_vals=np.linspace(0,d_in+2*d_out,1000)
r_vals=create_plot(X=z_vals, d_in=opt_coords[0],ω_0_in=ω_0_in,foc=foc,λ=λ)

# plot image
output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
# colorbar
color_mapper = LinearColorMapper( palette=Cividis256, low=0, high=1)
plot = figure(x_range=(-0.5,illust.shape[1]-0.5), y_range=(-illust.shape[0]/2,illust.shape[0]/2),plot_width=800, plot_height=150,
             title="locations of optimised mirror and chamber", x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
             tools=tools, active_drag=None, tooltips=[("(z , r)", "($x , $y)"), ("Intensity", "@image")])
plot.image(image=[illust], color_mapper=color_mapper,dh=[illust.shape[0]], dw=[illust.shape[1]], x=[-0.5], y=[-illust.shape[0]/2])

# show beam width
plot.line(z_vals,r_vals, line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
plot.line(z_vals,-r_vals, line_width=2,color="orange",line_dash="dashed",legend_label="beam profile")
# color bar
color_bar = ColorBar(color_mapper=color_mapper,label_standoff=12, location=(0,0))
plot.add_layout(color_bar, 'right')
# legend
plot.legend.label_text_font_size = '7pt'
plot.legend.background_fill_alpha = 0.5
plot.legend.padding=1
# show
print()
show(plot)

# Snippets

In [ ]:
Z_vals=np.arange(0,9)
Z_vals

In [ ]:
im=np.zeros((3,9))
im[2,Z_vals[-1]]

In [ ]:
if Z_vals[0]<0 or Z_vals[-1]>=im.shape[1]:
    print("yes")

In [ ]:
    # calculations
    just=(canvas.shape[0]-1)/2 #justify
    # very naive intensity for post reflection beam
    I_cell=(cal_trans_prof(d_in,ω_0_in,λ)/cal_trans_prof(-d_out,ω_0_out,λ)*ω_0_in/ω_0_out)**2
    image=np.copy(canvas)
    for z in tqdm(range(0,canvas.shape[1])):
        for r in range(0,canvas.shape[0]):
            if z<=d_in:
                ω_z=cal_trans_prof(z,ω_0_in,λ)
                image[r,z]=calculate_beam((r-just),ω_0_in,ω_z)
            else:
                ω_z=cal_trans_prof(z-round(d_in+d_out),ω_0_out,λ)
                image[r,z]=calculate_beam((r-just),ω_0_out,ω_z)*I_cell

In [ ]:
#Circle


# image=np.random.random((10,10))
image=np.zeros((11,11))
pos_chamber=5
pos=(np.int(np.round(image.shape[1])/2),np.int(np.round(pos_chamber)))
r_chamber=2
radius=np.int(np.round(r_chamber))
# check bounds
if (pos[0]+radius)>=image.shape[0] or (pos[0]-radius)<0 or (pos[1]+radius)>=image.shape[1] or (pos[1]-radius)<0:
  raise Exception("Sorry, chamber too large. Use larger canvas or decrease r_chamber.") 
# circle
for i in range(pos[0]-radius,pos[0]+radius+1):
    for j in range(pos[1]-radius,pos[1]+radius+1):
        if np.sqrt((pos[0]-i)**2+(pos[1]-j)**2)<=radius:
            image[i,j]=1

In [ ]:
plt.imshow(image)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot
output_notebook()
tools = "crosshair,pan,wheel_zoom,reset,save"
# colorbar
color_mapper = LinearColorMapper( palette=Cividis256, low=0, high=1)
plot = figure(x_range=(-0.5,image.shape[1]-0.5), y_range=(-image.shape[0]/2,image.shape[0]/2),plot_width=600, plot_height=300,
             title="beam intensity at given params", x_axis_label='optical axis z [mm]',y_axis_label='radial component r [mm]', 
             tools=tools, active_drag=None, tooltips=[("(z , r)", "($x , $y)"), ("Intensity", "@image")])
plot.image(image=[image], color_mapper=color_mapper,dh=[image.shape[0]], dw=[image.shape[1]], x=[-0.5], y=[-image.shape[0]/2])

show(plot)